<a href="https://colab.research.google.com/github/Mubashar-Bashir/Langchain_EcoSystem/blob/main/RAG_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.0 MB/s eta 0:00:00


In [2]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 3.5 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
userdata.get('google_api_key')
os.environ['GOOGLE_API_KEY'] = userdata.get('google_api_key')

In [5]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm.invoke("Agent Developer greet me!")

AIMessage(content='Hello there!  How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9849f3c9-3b15-4edc-96f4-8241f345c134-0', usage_metadata={'input_tokens': 6, 'output_tokens': 12, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#create embedding means numeric representation of text provided or document
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # lates model for embedding from google
em=embeddings.embed_query("H")
em[:3]


[0.0237551461905241, -0.060975316911935806, -0.0640849843621254]

In [14]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# os.environ['PINECONE_ENV'] = userdata.get('pinecone_env')
api_key = os.environ.get('PINECONE_API_KEY')


In [78]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "rag-langraph" # name must be lower case and ''
pc = Pinecone(api_key=api_key)

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)
print(index)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) #Vector Store

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '7a781966c6dc55cfed52a22e3ab8c21b', 'Date': 'Thu, 02 Jan 2025 21:03:53 GMT', 'Server': 'Google Frontend', 'Content-Length': '125', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Name must consist of lower case alphanumeric characters or '-'"},"status":400}


In [17]:
print(index)

In [50]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pprint import pprint
# Load documents
loader = TextLoader("/content/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs = text_splitter.split_documents(documents) # documents splitted
pprint(docs)

[Document(metadata={'source': '/content/document.txt'}, page_content='Here’s a concise **LangGraph Documentation** in text format based on its features, usage, and core concepts. You can save this as `langgraph_document.txt`:\n\n---\n\n# LangGraph Documentation\n\n## Overview\nLangGraph is a Python library designed for building stateful, multi-actor applications using large language models (LLMs). It is particularly useful for creating agent and multi-agent workflows with features like cycles, persistence, and human-in-the-loop support.\n\n---'),
 Document(metadata={'source': '/content/document.txt'}, page_content='---\n\n## Key Features\n1. **Cycles and Branching**:  \n   - Supports loops and conditional statements, enabling complex agent architectures.  \n   - Unlike DAG-based solutions, LangGraph allows dynamic flow control.  \n\n2. **Persistence**:  \n   - Automatically saves the state after each step.  \n   - Enables error recovery, human-in-the-loop workflows, and time travel.  \

In [19]:
!pip install tqdm

In [51]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
# vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content) #create embedding numeric

    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content)) #uuid or Hash doc_id = 22i3y12jdyuew****

    # Create metadata dictionary
    doc_metadata = {"source": doc.metadata["source"]}
    documents_list=[doc]
    # Upsert the vector with a unique ID and metadata
    # index.upsert(vectors=[(doc_id, vector, doc_metadata)]) # Include metadata in upsert
    vector_store.add_documents(documents=documents_list, id=doc_id)

100%|██████████| 8/8 [00:04<00:00,  1.97it/s]


In [55]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

# retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
# retriever = vector_store.as_retriever()


In [62]:
results = vector_store.similarity_search_with_score(
    "introduction to langgraph",
    k=4,
    # filter={"source": "/content/document.txt"}
    )
if results is not None:
    print(results)
else:
    print("No relevant documents found for the query.")
# for res in results:
#     print(f"*{res.page_content},[{res.metadata}]")
# print(results)

[(Document(id='df583ad0-f2b5-43c6-a9d0-a2ac4e46fc22', metadata={'source': '/content/document.txt'}, page_content='Here’s a concise **LangGraph Documentation** in text format based on its features, usage, and core concepts. You can save this as `langgraph_document.txt`:\n\n---\n\n# LangGraph Documentation\n\n## Overview\nLangGraph is a Python library designed for building stateful, multi-actor applications using large language models (LLMs). It is particularly useful for creating agent and multi-agent workflows with features like cycles, persistence, and human-in-the-loop support.\n\n---'), 0.796328187), (Document(id='0496c13e-c52b-4425-964e-82fcb2ac1402', metadata={'source': '/content/document.txt'}, page_content='---\n\nThis document provides a high-level overview of LangGraph. For more details, refer to the official GitHub repository and documentation.  \n\n--- \n\nSave this content as `langgraph_document.txt` for your reference! Let me know if you need further assistance.'), 0.78610

In [63]:
# Connect to your index
index = pc.Index(index_name)

# Fetch all vectors (documents) from the index
response = index.fetch(ids=[])  # Fetch all documents
vectors = response["vectors"]

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 02 Jan 2025 20:44:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '67', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '1617383947695068271', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No IDs provided for fetch query","details":[]}


In [76]:
# Connect to your index
index = pc.Index(index_name)

# Get all vector IDs in the index
all_ids = index.query(
    vector=[0.0] * 768,  # Dummy vector for querying all documents
    top_k=index.describe_index_stats()["namespaces"][""]["vector_count"],  # Get total number of vectors
    include_values=False,  # Only need IDs, not vectors
    include_metadata=False  # Only need IDs, not metadata
)["matches"]
all_ids = [match["id"] for match in all_ids]

# Fetch all vectors using the retrieved IDs
response = index.fetch(ids=all_ids)  # Fetch using all IDs
# pprint(response[])
vectors = response["vectors"]
pprint(vectors)

{'0496c13e-c52b-4425-964e-82fcb2ac1402': {'id': '0496c13e-c52b-4425-964e-82fcb2ac1402',
 'metadata': {'source': '/content/document.txt',
              'text': '---\n'
                      '\n'
                      'This document provides a high-level overview of '
                      'LangGraph. For more details, refer to the official '
                      'GitHub repository and documentation.  \n'
                      '\n'
                      '--- \n'
                      '\n'
                      'Save this content as `langgraph_document.txt` for your '
                      'reference! Let me know if you need further assistance.'},
 'values': [0.0640766621,
            -0.0170499068,
            -0.0460653082,
            -0.0213846676,
            0.0756447241,
            -0.0177212413,
            -0.00234630913,
            -0.0280652419,
            0.0427368246,
            0.0278452989,
            -0.042264957,
            0.0240725968,
            -0.0061941063,


In [66]:

# List to store IDs of documents without a `text` key
ids_need_to_delete = []

# Identify documents without a `text` key
for doc_id, doc in vectors.items():
    metadata = doc.get("metadata", {})  # Get metadata (default to empty dict if missing)
    if "text" not in metadata:
        ids_need_to_delete.append(doc_id)

# Print the IDs to delete
print("IDs to delete (missing 'text' key):")
pprint(ids_need_to_delete)


IDs to delete (missing 'text' key):
['-4679052358219276316',
 '8372902400715891593',
 '4469726895522432546',
 '3649838122007636025',
 '734098409067568971',
 '1242102442056521366',
 '-3228107013176541367',
 '-7537411710548254345']


In [67]:

# Delete the documents (optional)
if ids_need_to_delete:
    index.delete(ids=ids_need_to_delete)
    print(f"Deleted {len(ids_need_to_delete)} documents without a 'text' key.")
else:
    print("No documents without a 'text' key found.")

Deleted 8 documents without a 'text' key.


In [53]:
from langchain.chains import RetrievalQA
prompt = ''' user query is {query} and RAG documents context {res} '''
query = "introduction to langgraph"
vector_query = embeddings.embed_query(query)
results = vector_store.similarity_search(
    query,
    k=2,
    # filter={"source": "/content/document.txt"}
    )
prompt_template = prompt.format(query=query, res=results)

if results is not None:
    # response = llm.invoke(prompt_template+"Answer the query")
    response = results
    print(response)
else:
    print("No relevant documents found for the query.")


[]


In [34]:
query = "langgraph"
response = qa_chain.run(query)
print(response)

IndexError: list index out of range

In [36]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

query = "introduction to langgraph"
docs = retriever.(query) # Get relevant docs first

if not docs:
    print("No relevant documents found for the query.")
else:
    response = qa_chain.run(query)
    print(response)

No relevant documents found for the query.
